### Fit simple CNN
 + https://pytorch.org/tutorials/beginner/nn_tutorial.html

In [ ]:
%matplotlib inline

import sys
import logging
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch import nn
from torch import optim

sys.path.append('..')
import utils

logging.basicConfig(level=logging.INFO)

### Initialize computation device (CPU/GPU)

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Test relu

In [ ]:
x = range(-5,6)
plt.plot(x, F.relu(torch.tensor(x)).data.numpy(), 'bo-')
plt.grid(True)

### Get data

In [ ]:
data_dir = '../data'
x_train, y_train, x_valid, y_valid = utils.get_mnist(data_dir)

bs = 64

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs)

### Customize the dataloader

In [ ]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(DEVICE), y.to(DEVICE)

class CustomSizeDataLoader:
    """Serves batches with custom size."""
    def __init__(self, dl, func): 
        self.dl, self.func = dl, func
        
    def __len__(self): 
        return len(self.dl)
    
    def __iter__(self): 
        for b in self.dl: yield (self.func(*b))

train_dl = CustomSizeDataLoader(train_dl, preprocess)
valid_dl = CustomSizeDataLoader(valid_dl, preprocess)

### Define architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.adaptive_avg_pool2d(xb, 1)
        return xb.view(xb.size(0), -1)

### Set parameters

In [ ]:
torch.manual_seed(42)
epochs = 2
lr = 0.1
loss_func = F.cross_entropy

### Fit

In [ ]:
model = Net()
model.to(DEVICE)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

utils.fit(epochs, model, loss_func, opt, train_dl, valid_dl)

### Define the same architecture as above by using `nn.Sequential`

In [ ]:
class Lambda(nn.Module):
    """Used to define a custom layer (in `nn.Sequential`) from a given function."""
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)

torch.manual_seed(42)
model_seq = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1))
)

model_seq.to(DEVICE)
opt = optim.SGD(model_seq.parameters(), lr=lr, momentum=0.9)

utils.fit(epochs, model_seq, loss_func, opt, train_dl, valid_dl)

### Get number of model parameters

In [ ]:
print('#model parameters:', utils.count_parameters(model_seq))

# for example the third convolution has 16*10*9 weights and 10 biases